In [ ]:
import pandas as pd
from dateutil import parser
from geopy import Nominatim

import glob
import os

geolocator = Nominatim()

def locToCoords(location):
    coords = geolocator.geocode(location + " Charlottesville")
    longitude = coords.longitude
    latitude = coords.latitude
    return(latitude, longitude)

In [8]:
ticket_path = os.path.join('..','data','parking_tickets','Parking_Tickets.csv')
data = pd.read_csv(ticket_path, low_memory=0)

data = data[data['DateIssued'].str.contains('2017')].sort_values(by=['DateIssued'])